<a href="https://colab.research.google.com/github/wikibook/machine-learning/blob/2.0/jupyter_notebook/4.5_베르누이_나이브베이즈_스팸필터_실습.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리 임포트
실습에 필요한 라이브러리를 임포트합니다.

In [2]:
import numpy as np
import pandas as pd

# 베르누이 나이브베이즈를 위한 라이브러리를 임포트합니다
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB

# 모델의 정확도 평가를 위해 임포트합니다
from sklearn.metrics import accuracy_score
# sklearn 모델의 동일한 결과 출력을 위해 선언합니다.
np.random.seed(5)

# 문제 정의
베르누이 나이브베이즈 분류 모델을 사용하여 스팸 메일을 분류해보겠습니다.

# 데이터 수집
이번 실습에서는 간단한 스팸 메일 분류 실습을 위해 아래 이메일 타이틀과 스팸 여부가 있는 데이터를 사용하겠습니다.

In [3]:
email_list = [
                {'email title': 'free game only today', 'spam': True},
                {'email title': 'cheapest flight deal', 'spam': True},
                {'email title': 'limited time offer only today only today', 'spam': True},
                {'email title': 'today meeting schedule', 'spam': False},
                {'email title': 'your flight schedule attached', 'spam': False},
                {'email title': 'your credit card statement', 'spam': False}
             ]
df = pd.DataFrame(email_list)

In [4]:
df

,email title,spam
0,free game only today,True
1,cheapest flight deal,True
2,limited time offer only today only today,True
3,today meeting schedule,False
4,your flight schedule attached,False
5,your credit card statement,False


# 데이터 다듬기
sklearn의 베르누이 나이브베이즈 분류기(BernoulliNB)는 숫자만을 다루기 때문에,  
True와 False를 1과 0으로 치환하겠습니다.

In [5]:
df['label'] = df['spam'].map({True:1,False:0})
df

,email title,spam,label
0,free game only today,True,1
1,cheapest flight deal,True,1
2,limited time offer only today only today,True,1
3,today meeting schedule,False,0
4,your flight schedule attached,False,0
5,your credit card statement,False,0


In [6]:
# 학습에 사용될 데이터와 분류값을 나눕니다
df_x=df["email title"]
df_y=df["label"]

베르누이 나이브베이즈의 입력 데이터는 고정된 크기의 벡터로써, 0과 1로 구분된 데이터이여야 합니다.  
sklearn의 CountVectorizer를 사용하여 쉽게 구현할 수 있습니다.  
CountVectorizer는 입력된 데이터(6개의 이메일)에 출현된 모든 단어의 갯수만큼의 크기의 벡터를 만든 후,  
각각의 이메일을 그 고정된 벡터로 표현합니다.  
binary=True를 파라미터를 넘겨줌으로써, 각각의 이메일마다 단어가 한번 이상 출현하면 1, 출현하지 않을 경우 0으로 표시하게 합니다.

In [7]:
cv = CountVectorizer(binary=True)
x_traincv=cv.fit_transform(df_x)

아래의 행렬에서 볼 수 있듯, 데이터에서 총 17개의 단어가 발견되어, 각각의 이메일이 17개의 크기를 갖는 벡터로 표현되었습니다.  
또한 베르누이 나이브베이즈에 사용하기 위해 중복된 단어가 이메일 제목에 있더라도, 1로 표현된 것을 확인할 수 있습니다.  
limited time offer only today only today: [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0]

In [8]:
encoded_input=x_traincv.toarray()
encoded_input

array([[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]])

벡터로 인코딩된 이메일 제목에 어떤 단어들이 포함되어 있는 지 알고 싶을 경우, 아래의 명령어로 알 수 있습니다.

In [9]:
cv.inverse_transform(encoded_input[0].reshape(1,-1))

[array(['free', 'game', 'only', 'today'], dtype='<U9')]

벡터의 17개의 인덱스가 각각 무슨 단어를 의미하는 지 알고싶을 경우, 아래의 명령어를 통해 알 수 있습니다.

In [10]:
cv.get_feature_names_out()

['attached',
 'card',
 'cheapest',
 'credit',
 'deal',
 'flight',
 'free',
 'game',
 'limited',
 'meeting',
 'offer',
 'only',
 'schedule',
 'statement',
 'time',
 'today',
 'your']

# 베르누이 나이브베이즈 분류
베르누이 나이브베이즈로 스팸 메일을 분류해보도록 하겠습니다.  
BernoulliNB는 기본적으로 스무딩을 지원하므로, 학습데이터에 없는 단어가 테스트에 출현해도 분류를 이상없이 진행합니다.

In [11]:
# 학습 데이터로 베르누이 분류기를 학습합니다
bnb = BernoulliNB()
y_train=df_y.astype('int')
bnb.fit(x_traincv,y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [12]:
# 테스트 데이터 다듬기
test_email_list = [
                {'email title': 'free flight offer', 'spam': True},
                {'email title': 'hey traveler free flight deal', 'spam': True},
                {'email title': 'limited free game offer', 'spam': True},
                {'email title': 'today flight schedule', 'spam': False},
                {'email title': 'your credit card attached', 'spam': False},
                {'email title': 'free credit card offer only today', 'spam': False}
             ]
test_df = pd.DataFrame(test_email_list)
test_df['label'] = test_df['spam'].map({True:1,False:0})
test_x=test_df["email title"]
test_y=test_df["label"]
x_testcv=cv.transform(test_x)

# 테스트

In [13]:
predictions=bnb.predict(x_testcv)

# 정확도 (Accuarcy)

In [14]:
accuracy_score(test_y, predictions)

0.8333333333333334